# Forcing Generator Functions

This file contains programs that make the .nc forcing file that will be fed to OLLA. 

The first function makes the .nc, but requires the latter function to make the forcing on a minute-by-minute scale. In order to make this 
minute by minute forcing, we need minute by minute precipitation, 
which is why we have the last function.

This code largely borrows from make_forcing.py, originally written by Lucas Zeppetello.

Included functions: getForcingNetCDF, getRedNoise, getPrecip

In [2]:
import numpy as np 
import random

from netCDF4 import Dataset

random.seed(100) # Init seed for random number generator. If blank, the seed is the current time.

In [9]:
def getForcingNetCDF(tot_years, stats_array, filename):
    
    # This creates a netCDF file with two dimensions: the time (or
    # minutes) and the year (taken from tot_years). This will force
    # OLLA.
    
    mins_in_sum = 92*60*24 # minutes in a summer
    
    # Initialize arrays for forcing variables
    
    F_solar = np.zeros(shape=(mins_in_sum, tot_years))
    precip = np.zeros(shape=(mins_in_sum, tot_years))
    t_upper = np.zeros(shape=(mins_in_sum, tot_years))
    q_upper = np.zeros(shape=(mins_in_sum, tot_years))
    cloud_frac = np.zeros(shape=(mins_in_sum, tot_years))
    
    # Loop through initialized arrays and make "by the minute" forcing using 
    # getRedNoise(), below. So to be clear: this loop makes a year's worth of forcing,
    # while getRedNoise() makes forcing by the minute.
    
    for i in range(0, tot_years-1):
        minute_F_solar, minute_precip, minute_t_upper, minute_q_upper, minute_cloud_frac = getRedNoise(stats_array)
        F_solar[:,i] = minute_F_solar
        precip[:,i] = minute_precip
        t_upper[:,i] = minute_t_upper
        q_upper[:,i] = minute_q_upper
        cloud_frac[:,i] = minute_cloud_frac
        
    # Now to write the above forcing information to a .nc
    
    tmp_dataset = Dataset(filename+".nc", 'w', format='NETCDF3_64BIT')
    
    # Creating a "dimension" is equivalent to inializing a numpy array of shape
    # 'time'. You just have to add the dimensions to the .nc sequentially, as done below.
    
    tmp_dataset.createDimension('time', mins_in_sum)
    minutes = tmp_dataset.createVariable('time', 'i4', ('time', ))
    
    tmp_dataset.createDimension('summer_number', tot_years)
    summer_number = tmp_dataset.createVariable('summer_number', 'i4', ('summer_number', ))
    
    # Initialize vairables in the .nc that relate to forcing.
    
    F_solar_var = tmp_dataset.createVariable('F_solar', 'f4', ('time', 'summer_number', ))
    precip_var = tmp_dataset.createVariable('precip', 'f4', ('time', 'summer_number', ))
    t_upper_var = tmp_dataset.createVariable('t_upper', 'f4', ('time', 'summer_number', ))
    q_upper_var = tmp_dataset.createVariable('q_upper', 'f4', ('time', 'summer_number', ))
    cloud_frac_var = tmp_dataset.createVariable('cloud_frac', 'f4', ('time', 'summer_number', ))
    
    # Fill in values of the forcing variables in the .nc
    
    F_solar_var[:,:] = F_solar 
    precip_var[:,:] = precip
    t_upper_var[:,:] = t_upper
    q_upper_var[:,:] = q_upper
    cloud_frac_var[:,:] = cloud_frac
    
    # Voila!
    
    tmp_dataset.close()

In [3]:
def getRedNoise(stats_array):
    
    ### PARAMETER SETUP ###
    
    # Some parameters of getRedNoise are fed in from the SGP data, and thus are stored in 
    # stats_array.
    
    rad_anom_norm = stats_array[?]
    
    temp_mean = stats_array[?]
    temp_anom_norm = stats_array[?]
    
    hum_mean = stats_array[?]
    hum_anom_mean = stats_array[?]
    
    # Set to 0.6 in SLAM, not sure what they mean.
    
    r_temp = ?
    r_hum = ?
    r_rad = ?
    
    day_length = 15 # IN HOURS
    
    # I know how these are used, but not sure where they come from, or if they need to be 
    # sourced from the data.
    
    sw_limit = 900
    sw_average = 800
    sw_min = 200 
    
    ### CYCLE INITIALIZATION ###
    
    # The basic cycle is formed by taking a unit of free_time, then a unit of day_length,
    # then another unit of free_time. This reconstructs the full 24 hour day, and will be 
    # a useful structure going forward.
    
    free_time = (24 - day_length)*60/2. # In minutes!
    
    # Put the time of the day between, say, 9 and 5 and not from 0 to 8.
    
    day_time_shift = np.linspace(free_time, day_length*60 + free_time - 1, day_length*60)
    day_time = np.linspace(1, day_length*60, day_length*60)
    
    # Create radiation profiles as sin waves
    
    day_max = sw_limit * np.sin(np.pi * day_time * (day_length*60)**(-1))
    day_avg = sw_average * np.sin(np.pi * day_time * (day_length*60)**(-1))
    day_min = sw_min * np.sin(np.pi * day_time * (day_length*60)**(-1))
    
    # make "preday" and "postday," i.e., the time that does not include radiation
    
    pre_day = np.linspace(0, free_time - 1, free_time)
    post_day = np.linspace(day_length*60 + free_time, 24*60 - 1, free_time) # i.e., the time from sundown to the end of the day
    
    # make a 'no radiation' array
    
    no_rad = np.zeros(len(pre_day))
    
    # make an array of times that includes the entire day, from midnight to midnight (the next day)
    
    tot_day = np.hstack((pre_day, day_time_shift, post_day))
    tot_day_sw_max = np.hstack((no_rad, day_max, no_rad))
    tot_day_sw_avg = np.hstack((no_rad, day_avg, no_rad))
    tot_day_sw_min = np.hstack((no_rad, day_min, no_rad))
    
    # now we want to extend the above daily framework to the entire summer!
    
    all_summer_time = 92*24*60 # 92 days in summer, 24 hours in a day, 60 mins in an hour
    
    sw_max_sum = np.tile(tot_day_sw_max, 92) # make 92 days worth of the same radiation
    sw_avg_sum = np.tile(tot_day_sw_avg, 92)
    sw_min_sum = np.tile(tot_day_sw_min, 92)
    
    ### CREATE RED NOISE FORCING ###
    
    
    
    
    return F_solar, precip, t_up, q_up, cloud_frac

SyntaxError: invalid syntax (<ipython-input-3-bce2442f7516>, line 8)

In [8]:
def getInterpolate(array, new_length):
    old_time = np.linspace(0, new_length - 1, len(array))
    new_time = np.linspace(0, new_length - 1, new_length)
    new_array = np.interp(new_time, old_time, array)
    
    return new_array

In [ ]:
def getPrecip(stats_array):
    
    # Take the relevant statistics from the SGP data to tune our precip
    # events
    
    mean_precip = stats_array[?]
    std_precip = stats_array[?]
    
    indiv_event_mean = stats_array[?]
    
    # Generate a normal distribution for the cumulative rain?
    
    cumulative_rain = np.random.normal(mean_precip, std_precip)
    precip_events = []
    
    i = 0
    
    # Confused about this statement. What does it mean for the sum of the 
    # precipitation events to be less than the *distribution* of 
    # cumulative rain events?
    
    while cumulative_rain > np.sum(precip_events):
        tmp_precip = np.random.gamma(indiv_event_mean, scale=1.0) # why from a gamma dist?
        precip_events.append(tmp_precip)
        i += 1 # neat syntax :) 
    
    return(precip_events)